In [ ]:
import pandas as pd
#base_path = "C:/Users/99818854/Projetos/GitRep/adaptive_learning"
base_path = "/media/bruno/Arquivos/Desenvolvimento/NextQuestion"

In [ ]:
base = pd.read_csv(f"{base_path}/data/mastery.csv")
submit = pd.read_csv(f"{base_path}/data/Submit.csv", sep=";")